# Artifical Neural Network

# Part 1 - Data Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.12.0'

We do not want the first 3 columns, or the last one, exclude them (x) 3:-1 (starts as 0).  The y -1 is the last column, which is the answer.

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


Label endcoding of gender, take X column 2, emcde it bacck with transform

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encoding for geography, encode on the X column 1

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


split the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Build The ANN

Initialize with a varible - object

In [ ]:
from keras.engine.sequential import Sequential
ann = Sequential()

Add input layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Add second layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Add the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Train the ANN

Compile the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Train the ANN

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 26)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5909 - accuracy: 0.7310
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4684 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4391 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4265 - accuracy: 0.7970
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4166 - accuracy: 0.8010
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4039 - accuracy: 0.8087
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3925 - accuracy: 0.8152
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3831 - accuracy: 0.8244
Epoch 9/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3764 - accuracy: 0.8325
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3724 - accur

# Part 4 - Predictions and evaluating

testing a prediction, need to scale sc.transform

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 42ms/step
[[False]]


Predicting text results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1512   83]
 [ 197  208]]


0.86